# İmportların Yapılması
Gerekli kütüphaneleri ekleyelim

In [147]:
!pip install TurkishStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [148]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('turkish'))

from keras.models import load_model
from gensim.models import KeyedVectors

from keras.layers import Activation, SimpleRNN
from keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from keras.layers import MaxPooling1D
from tensorflow.keras.layers import Conv1D
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Verisetini Okunması
İlk olarak veri setini doğru formatta açalım.

In [149]:
"""
import urllib.request

url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz"
filename = "cc.tr.300.vec.gz"

# Dosyayı indirin
urllib.request.urlretrieve(url, filename)
"""

'\nimport urllib.request\n\nurl = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz"\nfilename = "cc.tr.300.vec.gz"\n\n# Dosyayı indirin\nurllib.request.urlretrieve(url, filename)\n'

In [150]:
dataset = pd.read_csv('datset5.csv')

# "sentence" ve "labels" sütunlarını ekle
dataset.columns = ['sentence', 'label']

Ön işleme aşamalarını yapalım

In [151]:
#Nan değer var mı bir kontrol edelim
nans = dataset.isna().sum().sum()
print("Toplam NaN değer sayısı:", nans)

# NaN değerlerinin bulunduğu satırların indekslerini bulma
nan_rows = dataset.isna().any(axis=1)
nan_row_indexes = list(dataset.index[nan_rows])
print("NaN değerleri içeren satır indeksleri:", nan_row_indexes)

# NaN değerleri içeren satırların değerlerini ekrana yazdırma
for i in nan_row_indexes:
    print("Satır", i, "değerleri:")
    print(dataset.iloc[i])

print("Toplam veri sayısı:", dataset.shape[0])

Toplam NaN değer sayısı: 38
NaN değerleri içeren satır indeksleri: [391, 392, 505, 506, 1032, 1033, 1180, 3593, 3594, 4207, 6617, 6932, 7334, 8102, 8395, 8949, 10172, 11164, 12089]
Satır 391 değerleri:
sentence    NaN
label       NaN
Name: 391, dtype: object
Satır 392 değerleri:
sentence    NaN
label       NaN
Name: 392, dtype: object
Satır 505 değerleri:
sentence    NaN
label       NaN
Name: 505, dtype: object
Satır 506 değerleri:
sentence    NaN
label       NaN
Name: 506, dtype: object
Satır 1032 değerleri:
sentence    NaN
label       NaN
Name: 1032, dtype: object
Satır 1033 değerleri:
sentence    NaN
label       NaN
Name: 1033, dtype: object
Satır 1180 değerleri:
sentence    NaN
label       NaN
Name: 1180, dtype: object
Satır 3593 değerleri:
sentence    NaN
label       NaN
Name: 3593, dtype: object
Satır 3594 değerleri:
sentence    NaN
label       NaN
Name: 3594, dtype: object
Satır 4207 değerleri:
sentence    NaN
label       NaN
Name: 4207, dtype: object
Satır 6617 değerleri:
sente

In [152]:
#Veri setindeki "sentence" sütunundaki noktalama işaretlerini kaldıralım.
dataset['sentence'] = dataset['sentence'].str.replace('[^\w\s]','', regex=True)

# Nan değerleri sil
dataset = dataset.dropna(inplace=False)
print("Toplam veri sayısı:", dataset.shape[0])

#dataset

Toplam veri sayısı: 14851


Maksimum kaç karakter var

In [153]:
""" Karakter sayısını ölçer
max_length = max(len(label) for label in dataset['label'])
max_label = dataset.loc[dataset['label'].str.len() == max_length]['label'].iloc[0]
print(f"En fazla karakter sayısı {max_length} olan veri: {max_label}")


max_length = max(len(label) for label in dataset['label'])
max_label = dataset.loc[dataset['label'].str.len() == max_length]['label'].iloc[0]
print(f"En fazla karakter sayısı {max_length} olan veri: {max_label}")
"""




#Etiket olarak 1 den fazla kelime var mı?
mask = dataset['label'].str.split().apply(lambda x: len(x)>1)
dataset[mask]

#dataset['sentence'] içinde 3 karakter az verileri bul
data = dataset[dataset['sentence'].apply(lambda x: len(x) < 3)]
data

#dataset içinde 3 karkaterden az verileri sil
dataset = dataset[dataset['sentence'].apply(lambda x: len(x) > 3)]
print("3 karakter az veriler silindi")
print(dataset)
print("dataset")


# Stop words'leri silme
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('turkish'))

num_of_stopwords = len(stop_words)

dataset['sentence'] = dataset['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

num_of_removed_words = sum(dataset['sentence'].apply(lambda x: num_of_stopwords - len(x.split())))

print(f"{num_of_removed_words} kelime stop words olarak silindi.")


"""
from snowballstemmer import TurkishStemmer
stemmer = TurkishStemmer()
def stem_sentence(sentence):
    # Split the sentence into words
    words = sentence.split()
    # Stem each word in the sentence and join them back
    stemmed_words = [stemmer.stemWord(word) for word in words]
    stemmed_sentence = ' '.join(stemmed_words)
    return stemmed_sentence
"""


from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()



# define the stem_sentence function
# Define the stem_cut function
def stem_cut(sentence):
    words = sentence.split()
    print(words)
    stemmed_words = ""
    words = sentence.split()
    for word in words:
        word= str(word)
        trnlp_word = stemmer.stem(word)
        print(trnlp_word)
        stemmed_words = stemmed_words + " " + trnlp_word
    stemmed_sentence = stemmed_words
    #print(stemmed_sentence)
    return stemmed_sentence

# Read the dataset into a DataFrame


# Apply the stem_cut function to the 'sentence' column
dataset['sentence'] = dataset['sentence'].apply(stem_cut)



print(dataset['sentence'])

dataset['sentence'] = dataset['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))



#Her etiketten 2000 adet al
dataset = dataset.groupby('label', as_index=False).apply(lambda x: x.sample(n=2000, random_state=42))

#dataset['sentence'] içinde 10 karakter az verileri bul
data = dataset[dataset['sentence'].apply(lambda x: len(x) < 20)]
data

label_mapping = {
    'OTHER': 0,
    'RACIST': 1,
    'PROFANITY': 2,
    'INSULT': 3,
    'SEXIST': 4
}

dataset['label'] = dataset['label'].map(label_mapping)


3 karakter az veriler silindi
                                                sentence      label
0                                            çürük dişli     INSULT
1      Bu adamın islama ve müslümanlara verdiği zarar...     RACIST
2                                   erkekler zora gelmez     SEXIST
3      Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...  PROFANITY
4           otomasyon sistemlerine doğrudan bağlanabilir      OTHER
...                                                  ...        ...
14865  Yeni bir spor denemek bana hem farklı bir akti...      OTHER
14866  Dostlarımı evimde ağırlamak bana büyük bir key...      OTHER
14867  Bu yıl kendime daha fazla zaman ayırıp kendimi...      OTHER
14868  Sabahları erken kalkıp güneşin doğuşunu izleme...      OTHER
14869  Yeni bir yemeği denemek benim için keşfetmekle...      OTHER

[14697 rows x 2 columns]
dataset


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Görüntülenen çıkış son 5000 satıra kısaltıldı.
git
bura
sen
cahil
bura
yer
yok
['Amına', 'kodumun', 'bok', 'çuvalı', 'bilir', 'seni']
Amına
kot
bok
çuva
bilir
sen
['n', 'neler', 'yaptığını']
n
ne
yaptık
['Seni', 'sevmek', 'büyük', 'hatamdı', 'senin', 'bir', 'piçin', 'varlığından', 'bile', 'utanıyorum']
Seni
sevmek
büyük
hata
sen
bir
piç
varlık
bil
utanıyor
['Ananı', 'avradını', 'sikeyim', 'senin', 'burada', 'işin', 'var']
Ananı
avrat
sike
sen
bura
iş
var
['İğrençsin', 'senin', 'yüzünden', 'işi', 'bitiremeyeceğiz']
İğrençsin
sen
yüz
iş
bitiremeyecek
['Senin', 'piçlere', 'burada', 'işim', 'yok', 'hadi', 'siktir', 'git']
Senin
piç
bura
iş
yok
hadi
sik
git
['Amına', 'koyduğumun', 'salağı', 'kadar', 'aptal', 'davranıyorsun']
Amına
koyduk
salak
kadar
aptal
davranıyor
['Orospu', 'çocuğu', 'başka', 'yerlerde', 'zaman', 'geçir', 'Burası', 'senin', 'uygun', 'değil']
Orospu
çocuk
başka
yer
zaman
geçir
Burası
sen
uygun
değil
['Seninle', 'uğraşacak', 'zamanım', 'yok', 'amına', 'koyayım']
Seninle
uğ

In [154]:
from TurkishStemmer import TurkishStemmer
stemmer = TurkishStemmer()
trnlp_word = stemmer.stem("okuldakilerden")
print(trnlp_word)

okul


Datasetimizi karıştırıyoruz

In [155]:
dataset = dataset.sample(frac=1, random_state=42)

In [156]:
# Verilerin %85'ini train, %15'ini test olarak ayır
train_data, test_data, train_labels, test_labels = train_test_split(
    dataset['sentence'], dataset['label'], test_size=0.15, random_state=42)

# Veri boyutlarını yazdır
print("Train veri boyutu:", len(train_data))
print("Test veri boyutu:", len(test_data))

Train veri boyutu: 8500
Test veri boyutu: 1500


In [157]:
#Birkaç rastgele deneme yapalım
"""
while range(1,5):
  x = int(input("Bir sayı gir"))
  print(train_data.iloc[x])
  print(train_labels.iloc[x])
"""

'\nwhile range(1,5):\n  x = int(input("Bir sayı gir"))\n  print(train_data.iloc[x])\n  print(train_labels.iloc[x])\n'

# Toplam kelime sayısını bulup Tokenize yapalım

In [158]:
#Veri setindeki tüm cümleleri bir listeye dönüştürün:
sentences = list(dataset['sentence'])

#Tüm cümleleri tek bir string haline getirin ve küçük harflere çevirin:
all_text = ' '.join(sentences).lower()

##Stringi kelimelere ayırın:
words = all_text.split()


#Farklı kelime sayısını bulun:
unique_words = set(words)
num_unique_words = len(unique_words)
print("Toplam farklı kelime sayısı:", num_unique_words)


Toplam farklı kelime sayısı: 11485


In [159]:

from collections import Counter

# Tüm kelimeleri bir liste halinde alalım
all_words = ' '.join(list(train_data)).split()

# Her bir kelimenin kaç kez geçtiğini bulalım
word_counts = Counter(all_words)
""" 
# Kelimelerin kaç kez geçtiğini yazdıralım
for word, count in word_counts.items():
    print(f"'{word}' kelimesi {count} kez geçiyor.")
"""

#Geçme sayılarına göre büyükten küçüğe doğru sırala
word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))
print(word_counts)

{'bir': 930, 'sen': 772, 'kat': 433, 'erkek': 367, 'insan': 308, 'var': 261, 'am': 241, 'kadar': 234, 'Senin': 225, 'ben': 221, 'kız': 215, 'iş': 205, 'ev': 190, 'değil': 190, 'siker': 190, 'yer': 187, 'göt': 178, 'çocuk': 174, 'zaman': 170, 'iyi': 163, 'olur': 163, 'yok': 163, 'sike': 159, 'Kadınlar': 158, 'Erkekler': 154, 'Bu': 149, 'sik': 143, 'bil': 138, 'anan': 137, 'kent': 134, 'olmaz': 120, 'ada': 119, 'dedik': 119, 'Kadın': 117, 'olan': 116, 'on': 116, 'hayat': 113, 'bun': 112, 'olma': 112, 'sade': 107, 'mi': 105, 'olduk': 104, 'yapar': 104, 'orosp': 99, 'ülke': 96, 'iç': 96, 'baş': 89, 'kar': 89, 'olarak': 88, 'san': 86, 'git': 85, 'ediyor': 84, 'adam': 82, 'onlar': 81, 'ols': 79, 'al': 79, 'yüz': 78, 'sonr': 76, 'eder': 74, 'bi': 74, 'suriye': 73, 'ol': 73, 'yemek': 72, 'çingen': 69, 'gör': 68, 'kürt': 68, 'bak': 67, 'bura': 66, 'yapma': 66, 'kadın': 65, 'Sen': 64, 'yol': 63, 'Erkek': 63, 'söz': 63, 'ara': 62, 'gün': 61, 'kafa': 61, 'böyl': 61, 'sahip': 61, 'ağz': 60, 'son': 

In [160]:
#Belirli bir eşik değeri üzerinde en az 2 defa geçen kelimelerin sayısını bul:
word_count = {}
threshold = 2

# Veri setindeki her bir cümle için kelimeleri say
for sentence in dataset['sentence']:
    words = sentence.split()
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

# Eşik değerinin üzerinde en az 2 defa geçen kelimeleri say
count = 0
for key, value in word_count.items():
    if value > threshold:
        count += 1

print("En az 2 defa geçen kelime sayısı:", count)

En az 2 defa geçen kelime sayısı: 3248


In [161]:
# Eşik değerinin üzerinde en az 2 defa geçen kelimeleri num_words yap gerisi Tokenize işlemine alma
#Kendi tokenimi yapacağımdan bu kısmı kullanmıyorum

toplam_benzersiz = 18941
num_words = toplam_benzersiz
tokenizer = Tokenizer(num_words=num_words)
target = dataset['sentence'].values.tolist()
tokenizer.fit_on_texts(target)
tokenizer.word_index


{'bir': 1,
 'sen': 2,
 'kat': 3,
 'erkek': 4,
 'insan': 5,
 'var': 6,
 'am': 7,
 'kız': 8,
 'ben': 9,
 'kadar': 10,
 'senin': 11,
 'iş': 12,
 'ev': 13,
 'siker': 14,
 'değil': 15,
 'çocuk': 16,
 'yer': 17,
 'kadın': 18,
 'olur': 19,
 'zaman': 20,
 'göt': 21,
 'iyi': 22,
 'yok': 23,
 'erkekler': 24,
 'kadınlar': 25,
 'sike': 26,
 'bu': 27,
 'sik': 28,
 'bil': 29,
 'kent': 30,
 'anan': 31,
 'dedik': 32,
 'olmaz': 33,
 'ada': 34,
 'on': 35,
 'olma': 36,
 'mi': 37,
 'olan': 38,
 'bun': 39,
 'sade': 40,
 'hayat': 41,
 'olduk': 42,
 'adam': 43,
 'yapar': 44,
 'ülke': 45,
 'iç': 46,
 'orosp': 47,
 'olarak': 48,
 'kürt': 49,
 'san': 50,
 'baş': 51,
 'kar': 52,
 'ediyor': 53,
 'al': 54,
 'git': 55,
 'gavur': 56,
 'yüz': 57,
 'türk': 58,
 'onlar': 59,
 'ol': 60,
 'suriye': 61,
 'bi': 62,
 'arap': 63,
 'çingen': 64,
 'sonr': 65,
 'ols': 66,
 'yemek': 67,
 'eder': 68,
 'bura': 69,
 'gör': 70,
 'bak': 71,
 'yapma': 72,
 'aynı': 73,
 'konu': 74,
 'söz': 75,
 'böyl': 76,
 'gün': 77,
 'sahip': 78,
 'y

In [162]:
#train_data, test_data, train_labels, test_labels
"""
import tensorflow as tf
import numpy as np

def Word2Token(train_data, model, max_len=27):
    sentence_tokens = []
    for sentence in train_data:
        tokens = sentence.split()
        tokens = [token if token in model.vocab else "<UNK>" for token in tokens]
        sentence_tokens.append(tokens)

    # Convert tokens to indices
    word2idx = {word: i for i, word in enumerate(model.vocab)}
    sentence_indices = [[word2idx[token] if token in word2idx else word2idx['<UNK>'] for token in sentence] for sentence in sentence_tokens]

    # Pad sequences
    sentence_indices = tf.keras.preprocessing.sequence.pad_sequences(sentence_indices, maxlen=max_len, padding='post', truncating='post')

    # Embed sequences
    embedding_layer = tf.keras.layers.Embedding(input_dim=model.vectors.shape[0], output_dim=model.vectors.shape[1], weights=[model.vectors], trainable=False)
    train_data_tokens = embedding_layer(sentence_indices)
    
    return train_data_tokens.numpy()





#33 dakika süren fonksiyon

# Veri setindeki her cümle için tokenleri oluştur
def Word2Token(train_data, model, max_len=27):
    sentence_tokens = []
    for sentence in train_data:
        tokens = sentence.split()
        tokens = [token if token in model.vocab else "<UNK>" for token in tokens]
        sentence_tokens.append([model.get_word_vector(token) for token in tokens])

    # Tokenlerin boyutlarını hizala ve numpy dizisi olarak değiştir
    train_data_tokens = np.zeros((len(sentence_tokens), max_len, model.get_dimension()))
    for i in range(len(sentence_tokens)):
        for j in range(min(len(sentence_tokens[i]), max_len)):
            train_data_tokens[i][j] = sentence_tokens[i][j]

    return train_data_tokens




model_path = '/content/drive/MyDrive/cc.tr.300.vec.gz'
model = KeyedVectors.load_word2vec_format(model_path, binary=True, encoding='iso-8859-1')

train_data_tokens = Word2Token(train_data,model)
test_data_tokens = Word2Token(test_data,model)

"""
#  eski kısım

train_data_tokens = tokenizer.texts_to_sequences(train_data)
test_data_tokens = tokenizer.texts_to_sequences(test_data)


In [163]:
#Tokenize edilmiş cümleyi test et
print(train_data.iloc[801])
print(train_data_tokens[801])

word_map = tokenizer.word_index
word_map = {k:v for k,v in word_map.items() if v in train_data_tokens[801]}
result = [word for word, index in word_map.items()]
print(result)

Kadınlar iyi şef olamaz
[25, 22, 2343, 123]
['iyi', 'kadınlar', 'olamaz', 'şef']


In [164]:
#Bu kod öncelikle, eğitim ve test veri setlerindeki tüm cümlelerin belirteçlere (tokens) ayrıldığı x_train_tokens ve x_test_tokens listelerini birleştirir. Daha sonra her bir cümle için belirteç sayısını (yani cümlenin uzunluğunu) hesaplar ve num_tokens adlı bir numpy dizisinde saklar.
num_tokens = [len(tokens) for tokens in train_data_tokens + test_data_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([6, 5, 2, ..., 2, 3, 2])

In [165]:
#num_tokens listesindeki sayıların ortalamasını döndürür. Yani, tüm cümlelerin belirli bir dönüştürme işlemi sonucu elde edilen toplam kelime sayısının ortalamasını hesaplar.
np.mean(num_tokens)

5.2533

In [166]:
#verilen listenin en büyük elemanını (maximum) döndürür. 
np.max(num_tokens)

27

In [167]:
# np.argmax(num_tokens) en uzun veri örneğinin dizinini (indeksini) verir.
np.argmax(num_tokens)

5826

In [168]:
#Bu kod satırları veri setindeki tüm tokenlerin uzunluklarının ortalaması (mean) ve standart sapmasını (std) hesaplar ve daha sonra maksimum uzunluk sınırını belirlemek için ortalama ve iki kat standart sapmayı toplar. Son olarak, max_tokens, bir tam sayıya dönüştürülür ve maksimum token uzunluğu olarak belirlenir. Bu yaklaşım, veri setindeki token uzunluklarının genel dağılımına dayanarak bir üst sınır belirler.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

10

In [169]:
#Bu kod satırı, belirlediğimiz max_tokens sayısı sınırı içinde kalan token oranını hesaplar. num_tokens listesindeki her bir sayı, eğitim ve test verilerindeki bir örnek için kelime sayısını temsil eder. max_tokens sınırına uymayan örnekler atılacak ve bu satır bu örneklerin oranını verir.
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9223

In [170]:
#Bu satır, eğitim veri setindeki her bir örneğin kelime dizilimlerini, maxlen parametresi tarafından belirtilen maksimum kelime sayısına kadar kırpıyor veya sıfırlıyor. pad_sequences() fonksiyonu, kelime dizilerini verilen maksimum kelime sayısı kadar doldururken, daha kısa kelime dizilerini sıfırlar ve daha uzun kelime dizilerini kırparak, tüm örneklerin aynı boyutta olmasını sağlar.
#train_data, test_data, train_labels, test_labels
x_train_pad = pad_sequences(train_data_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(test_data_tokens, maxlen=max_tokens)

#x_train_pad'in şekli, önceki şekilden farklı olarak (örnek sayısı, en fazla kelime sayısı) şeklindedir. Her bir örnek en fazla "max_tokens" kelime içerecek şekilde yeniden boyutlandırılmıştır.
x_train_pad.shape

(8500, 10)

In [171]:
#x_test_pad'in şekli, önceki şekilden farklı olarak (örnek sayısı, en fazla kelime sayısı) şeklindedir. Her bir örnek en fazla "max_tokens" kelime içerecek şekilde yeniden boyutlandırılmıştır.
x_test_pad.shape

(1500, 10)

In [172]:
#Eğitim için bir model oluşturuyorsunuz, ancak henüz hangi katmanları kullanacağınızı belirtmediniz. 
model = Sequential()


"""
x_train_pad.shape sonucu, 8500 örnekten oluşan bir eğitim setinin her bir örneğinin maksimum 12 kelime uzunluğunda olduğunu gösteriyor. Bu bilgiye dayanarak, embedding_size değerini belirlemek zor. 
Genellikle, embedding_size'ın num_words parametresine yakın olması önerilir. Ancak bu bir kural değil, ve bu değeri belirlemek için farklı yöntemler kullanılabilir. 
Örneğin, deneme yanılma yöntemi kullanarak farklı embedding boyutlarını (örneğin, 32, 64, 128) deneyebilirsiniz ve hangisinin en iyi sonuçları verdiğine karar verebilirsiniz.
"""
embedding_size = 128

In [173]:
"""model = Sequential([
    Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens)])
  """



'model = Sequential([\n    Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens)])\n  '

In [174]:
vocab_size = len(tokenizer.word_index)+1


model.add(Embedding(input_dim=4000,
output_dim=50,
input_length=max_tokens,
name='embedding_layer'))

model.add(SimpleRNN(units=16, return_sequences=True))
model.add(SimpleRNN(units=8, return_sequences=True))
model.add(SimpleRNN(units=5))
model.add(Dense(5, activation='softmax'))


"""
model.add(CuDNNGRU(units=16, return_sequences=True))
model.add(CuDNNGRU(units=8, return_sequences=True))
model.add(CuDNNGRU(units=5))
model.add(Dense(5, activation='softmax'))
"""

"""
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=1))

model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='sigmoid'))
"""
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 10, 50)            200000    
_________________________________________________________________
module_wrapper_12 (ModuleWra (None, 10, 16)            1072      
_________________________________________________________________
module_wrapper_13 (ModuleWra (None, 10, 8)             200       
_________________________________________________________________
module_wrapper_14 (ModuleWra (None, 5)                 70        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 30        
Total params: 201,372
Trainable params: 201,372
Non-trainable params: 0
_________________________________________________________________


In [175]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [176]:



##train_data, test_data, train_labels, test_labels
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


y_train_onehot = to_categorical(train_labels)

model.fit(x_train_pad, y_train_onehot, epochs=25, batch_size=50)


Epoch 1/25
170/170 [==============================] - 8s 30ms/step - loss: 1.4335 - accuracy: 0.3891
Epoch 2/25
170/170 [==============================] - 4s 21ms/step - loss: 1.0040 - accuracy: 0.6911
Epoch 3/25
170/170 [==============================] - 4s 22ms/step - loss: 0.6488 - accuracy: 0.8494
Epoch 4/25
170/170 [==============================] - 5s 30ms/step - loss: 0.4114 - accuracy: 0.9152
Epoch 5/25
170/170 [==============================] - 4s 21ms/step - loss: 0.2918 - accuracy: 0.9401
Epoch 6/25
170/170 [==============================] - 4s 21ms/step - loss: 0.2178 - accuracy: 0.9546
Epoch 7/25
170/170 [==============================] - 4s 25ms/step - loss: 0.1710 - accuracy: 0.9644
Epoch 8/25
170/170 [==============================] - 4s 26ms/step - loss: 0.1405 - accuracy: 0.9698
Epoch 9/25
170/170 [==============================] - 4s 21ms/step - loss: 0.1191 - accuracy: 0.9735
Epoch 10/25
170/170 [==============================] - 4s 21ms/step - loss: 0.1112 - accura

In [177]:
test_labels = to_categorical(test_labels)
result = model.evaluate(x_test_pad, test_labels)

47/47 [==============================] - 1s 5ms/step - loss: 1.1807 - accuracy: 0.7327


In [178]:

"""
# Yanlış tahmin edilen cümlelerin indekslerini bul
misclassified = np.where(predicted_labels != test_labels)[0]

# Yanlış tahmin edilen cümlelerin DataFrame'deki indekslerini filtreleyin
misclassified_df = dataset.iloc[misclassified]
for index, row in misclassified_df.iterrows():
    print("Cümle: ", row['sentence'])
    print("Gerçek Etiket: ", label_names[row['label']])
    test_sentence =  row['sentence']
    test_sentence = test_sentence.lower().replace('[^\w\s]','')
    test_sentence = ' '.join([word for word in test_sentence.split() if word not in (stop_words)])
    test_sequences = tokenizer.texts_to_sequences([test_sentence])
    test_padded = pad_sequences(test_sequences, maxlen=max_tokens)


    # Tahmin edilen sınıfı içeren numpy dizisini oluştur
    #print(x_test_pad)
    prediction = model.predict(test_padded)
    #print(prediction)
    predicted_label = np.argmax(prediction)
    #print(predicted_label)
    # Tahmin edilen sınıfın adını yazdırın


    
    predicted_class_name = label_names[predicted_label]
    print("Tahmin Edilen Etiket: ", predicted_class_name)
    print()

"""

'\n# Yanlış tahmin edilen cümlelerin indekslerini bul\nmisclassified = np.where(predicted_labels != test_labels)[0]\n\n# Yanlış tahmin edilen cümlelerin DataFrame\'deki indekslerini filtreleyin\nmisclassified_df = dataset.iloc[misclassified]\nfor index, row in misclassified_df.iterrows():\n    print("Cümle: ", row[\'sentence\'])\n    print("Gerçek Etiket: ", label_names[row[\'label\']])\n    test_sentence =  row[\'sentence\']\n    test_sentence = test_sentence.lower().replace(\'[^\\w\\s]\',\'\')\n    test_sentence = \' \'.join([word for word in test_sentence.split() if word not in (stop_words)])\n    test_sequences = tokenizer.texts_to_sequences([test_sentence])\n    test_padded = pad_sequences(test_sequences, maxlen=max_tokens)\n\n\n    # Tahmin edilen sınıfı içeren numpy dizisini oluştur\n    #print(x_test_pad)\n    prediction = model.predict(test_padded)\n    #print(prediction)\n    predicted_label = np.argmax(prediction)\n    #print(predicted_label)\n    # Tahmin edilen sınıfın adı

In [179]:
"""
# Tahmin edilen sınıfı içeren numpy dizisini oluştur
predicted_classes = model.predict_classes(x_test_pad)

# Gerçek sınıfı içeren numpy dizisini oluştur
true_classes = np.argmax(test_labels, axis=1)

# Yanlış tahmin edilen cümlelerin indekslerini bul
misclassified = np.where(predicted_classes != true_classes)[0]

# Yanlış tahmin edilen cümlelerin DataFrame'deki indekslerini filtreleyin
misclassified_df = dataset.iloc[misclassified]

# Yanlış tahmin edilen cümleleri ekrana yazdırın
for index, row in misclassified_df.iterrows():
    print("Cümle: ", row['sentence'])
    print("Gerçek Etiket: ", label_names[row['label']])
    predicted_class_name = label_names[predicted_label]
    print("Tahmin Edilen Etiket: ", predicted_class_name)
    print()

  """

'\n# Tahmin edilen sınıfı içeren numpy dizisini oluştur\npredicted_classes = model.predict_classes(x_test_pad)\n\n# Gerçek sınıfı içeren numpy dizisini oluştur\ntrue_classes = np.argmax(test_labels, axis=1)\n\n# Yanlış tahmin edilen cümlelerin indekslerini bul\nmisclassified = np.where(predicted_classes != true_classes)[0]\n\n# Yanlış tahmin edilen cümlelerin DataFrame\'deki indekslerini filtreleyin\nmisclassified_df = dataset.iloc[misclassified]\n\n# Yanlış tahmin edilen cümleleri ekrana yazdırın\nfor index, row in misclassified_df.iterrows():\n    print("Cümle: ", row[\'sentence\'])\n    print("Gerçek Etiket: ", label_names[row[\'label\']])\n    predicted_class_name = label_names[predicted_label]\n    print("Tahmin Edilen Etiket: ", predicted_class_name)\n    print()\n\n  '

In [180]:
#testin başarı oranı
result[1]

0.7326666712760925

In [181]:
y_pred = model.predict(x=x_test_pad[0:1400])
y_pred



array([[1.39808655e-02, 9.74354744e-01, 1.08381724e-02, 1.42400968e-04,
        6.83767546e-04],
       [6.97362120e-04, 1.28729886e-03, 1.96984115e-06, 2.52509909e-03,
        9.95488226e-01],
       [3.57040274e-03, 1.17768080e-03, 9.94176865e-01, 1.07283378e-03,
        2.21176697e-06],
       ...,
       [5.15889551e-05, 4.06570323e-02, 3.82890040e-03, 9.15463567e-01,
        3.99988294e-02],
       [5.88465631e-01, 7.30556436e-04, 4.09156144e-01, 1.46929210e-03,
        1.78386850e-04],
       [8.59775813e-04, 9.92766380e-01, 9.48492961e-04, 1.28877664e-03,
        4.13663313e-03]], dtype=float32)

In [182]:
# Önce sınıf olasılıklarını sınıf etiketlerine dönüştürün
print(y_pred)

y_pred_labels = np.argmax(y_pred, axis=1)
print(y_pred_labels)
# Test etiketlerini sınıf etiketlerine dönüştürün
y_test_labels = np.argmax(test_labels, axis=1)
print(y_test_labels)
# Yanlış tahmin edilen örneklerin indekslerini ve sınıf etiketlerini bulun
dizi = []
incorrect = np.nonzero(y_pred_labels != y_test_labels)[0]



[[1.39808655e-02 9.74354744e-01 1.08381724e-02 1.42400968e-04
  6.83767546e-04]
 [6.97362120e-04 1.28729886e-03 1.96984115e-06 2.52509909e-03
  9.95488226e-01]
 [3.57040274e-03 1.17768080e-03 9.94176865e-01 1.07283378e-03
  2.21176697e-06]
 ...
 [5.15889551e-05 4.06570323e-02 3.82890040e-03 9.15463567e-01
  3.99988294e-02]
 [5.88465631e-01 7.30556436e-04 4.09156144e-01 1.46929210e-03
  1.78386850e-04]
 [8.59775813e-04 9.92766380e-01 9.48492961e-04 1.28877664e-03
  4.13663313e-03]]
[1 4 2 ... 3 0 1]
[1 4 2 ... 3 4 3]


<ipython-input-182-ffa03de85d47>:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  incorrect = np.nonzero(y_pred_labels != y_test_labels)[0]


In [183]:
idx = incorrect[0]
idx

0

In [ ]:
# Kullanıcıdan cümle isteme

while(True):
  test_sentence = input("Bir cümle girin: ")

  # Cümleyi preprocess edip, tokenize etme ve padding yapma
  test_sentence = test_sentence.lower().replace('[^\w\s]','')
  test_sentence = ' '.join([word for word in test_sentence.split() if word not in (stop_words)])
  test_sequences = tokenizer.texts_to_sequences([test_sentence])
  test_padded = pad_sequences(test_sequences, maxlen=max_tokens)
  print(test_padded)
  # Etiketi tahmin etme

  prediction = model.predict(test_padded)
  print(prediction)
  predicted_label = np.argmax(prediction)
  print(prediction)
  # Tahmin edilen etiketi yazdırma
  label_names = ['Normal', 'Irkçı', 'Küfür', 'Hakaret', 'Cinsiyetçi']
  print("Tahmin edilen etiket:", label_names[predicted_label])

